In [1]:
import sys
import os
from typing import List
sys.path.insert(0, os.path.abspath('..'))

%load_ext autoreload
%autoreload 2

This Notebooks demonstrates how we can use RasterTransformer and MultiRasterSource.

We can:
- Apply custom normalizations using RasterTransformer
- Merge multiple images (e.g. S1 and S2) using MultiRasterSource
- Create Indices (e.g. NDWI) using RasterTransformer and merge tusing MultiRasterSource

In [2]:
from typing import Optional
import numpy as np

from rastervision.core.data import RasterioSource, MultiRasterSource
from rastervision.core.data.raster_transformer.raster_transformer import RasterTransformer
from rastervision.core.box import Box

image_uri = "https://storage.googleapis.com/sand_mining/labels/Betwa_Hamirpur_79-81_25-91/rgb/Betwa_Hamirpur_79-81_25-91_2022-03-22_rgb.tif"

class CustomTransformer(RasterTransformer):
    """
    Dummy transformer for demonstration
    """
    def __init__(self):
        pass

    def transform(self, chip: np.ndarray,
                  channel_order: Optional[list] = None) -> np.ndarray:
        summed = chip.sum(axis=2)
        return np.expand_dims(summed, axis=2)


img_original = RasterioSource(
    image_uri,
)
img_custom = RasterioSource(
    image_uri,
    raster_transformers=[CustomTransformer()]
)

# We can merge them into one RasterSource
combined_img = MultiRasterSource(
    [img_original, img_custom],
    force_same_dtype=True
)


dummy_bbox = Box(0,0,150,150)
print(f"Chip from original    image has shape {img_original.get_chip(dummy_bbox).shape}")
print(f"Chip from transformed image has shape {img_custom.get_chip(dummy_bbox).shape}")
print(f"Chip from combined    image has shape {combined_img.get_chip(dummy_bbox).shape}")


/home/tboehnel/.conda/envs/rastervision/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-27 14:19:59:rastervision.pipeline.file_system.utils: INFO - Using cached file /data/tmp/cache/http/storage.googleapis.com/sand_mining/labels/Betwa_Hamirpur_79-81_25-91/rgb/Betwa_Hamirpur_79-81_25-91_2022-03-22_rgb.tif.
2023-07-27 14:19:59:rastervision.pipeline.file_system.utils: INFO - Using cached file /data/tmp/cache/http/storage.googleapis.com/sand_mining/labels/Betwa_Hamirpur_79-81_25-91/rgb/Betwa_Hamirpur_79-81_25-91_2022-03-22_rgb.tif.


Chip from original    image has shape (150, 150, 3)
Chip from transformed image has shape (150, 150, 1)
Chip from combined    image has shape (150, 150, 4)
